# API Twitter

In [ ]:
import tweepy as tw
import requests

In [ ]:
consumer_secret = 'iwefrBeyulr5ASaLxvq2yH0gGzTsVUj0QR8ilp6poYsKah9beR'
consumer_key = 'mAp7jaz6G0ruPUZd4EqQ0j9lI'
access_token = '1490812061919625223-O4tGXjfO4JBEMjx62Cq7iTBoxWSiar'
access_token_secret = '3VXjT0mdvEoalJd61yhftJLgve8vhvxwG5if9nBkzpolg'

In [ ]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)

In [ ]:
auth.set_access_token(access_token, access_token_secret)

In [ ]:
api = tw.API(auth)

In [ ]:
tweet = api.update_status("Python e Twitter #alura")

In [ ]:
tweet

# Pesquisando Tweets

In [ ]:
tweets = tw.Cursor(api.search, q="data science",since='2019-01-01',lang='pt').items(10)

In [ ]:
for tweet in tweets:
  print(tweet.text)
  print("Name: " + tweet.user.name)
  print("Screen name: " + tweet.user.screen_name)
  print("Localização: " + tweet.user.location)
  print('----------------------')
  print('\n\n')

In [ ]:
famosos = ['cauareymond','aguiarthur','ivetesangalo','ClaudiaLeitte','neymarjr','BruMarquezine','mariruybarbosa',
'FePaesLeme','Tatawerneck','FlaviaAleReal','julianapaes','dedesecco','SabrinaSato','ahickmann','gusttavo_lima','Anitta',
'CarolCastroReal','gio_antonelli','maisa','cleooficial','gewbank','taisdeverdade','otaviano','bernardipaloma',
'IngridGuimaraes','olazaroramos','GalisteuOficial','debranascimento','FioMattheis','moalfradique','Nandacostareal']

In [ ]:
len(famosos)

In [ ]:
for famoso in famosos:
    tweets = tw.Cursor(api.search, q=famoso,since='2019-01-01').items(20)

    print('Autor: ', famoso)
    print('Imagens postadas:')
    for tweet in tweets:
        if 'media' in tweets.entities:
            print(tweet.entities['media'][0]['media_url'])
    print('----------------------')
    print('\n\n')


# Streaming

In [ ]:
class MinhaStreamListener(tw.StreamListener):
    
    def on_status(self, status):
        print("Name: " + tweet.user.name)
        print("Screen name: " + tweet.user.screen_name)
        print("Texto: " + tweet.text)
        print('----------------------')
        print('\n\n')


In [ ]:
minhaStream = tw.Stream(auth = auth, listener=MinhaStreamListener())

In [ ]:
minhaStream.filter(track=famosos)

# Azure Computer Vision

In [ ]:
!pip install azure-cognitiveservices-vision-computervision

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials

In [ ]:
key1 = '6f3af9ccf122420bb41957b932d10167'
key2 = '3a089c7bfba64452a8896286756170b9'

credencias = CognitiveServicesCredentials(key1)

In [ ]:
uri = 'https://cvaluratwitter.cognitiveservices.azure.com/'
client = ComputerVisionClient(uri, credencias)

In [ ]:
client.api_version

![](https://pbs.twimg.com/media/ECx6hK-WwAAPzeE.jpg)

In [ ]:
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes

url = 'https://pbs.twimg.com/media/ECx6hK-WwAAPzeE.jpg'

analise_de_imagem = client.analyze_image(url, visual_features = [VisualFeatureTypes.tags])

In [ ]:
for tag in analise_de_imagem.tags:
    print(tag)

In [ ]:
analise_de_celebridades = client.analyze_image_by_domain("celebrities", url, "en")

In [ ]:
for celebridade in analise_de_celebridades.result["celebrities"]:
    print(celebridade['name'])
    print(celebridade['confidence'])

In [ ]:
descricacao = client.describe_image(url,3,"en")
descricacao

In [ ]:
for caption in descricacao.captions:
    print(caption.text)
    print(caption.confidence)

# Streaming + Vision API

In [ ]:
import json

In [ ]:
api.get_user('jcalvesol').id_str

In [ ]:
famosos_id = []
for famoso in famosos:
    famoso_id = api.get_user(famoso).id_str
    famosos_id.append(famoso_id)

In [ ]:
class MinhaStreamListener(tw.StreamListener):
    try:
        def on_status(self, status):
            print("Nome: " + tweet.user.name)
            print("usuário: " + tweet.user.screen_name)
            print("Texto: " + tweet.text)
            

            if 'media' in status.entites:
                url = status.entities['media'][0]['media_url']
                print("URL :", url)

                analise_de_celebridades = client.analyze_image_by_domain("celebrities", url, "en")

                lista_celebridades = [celebridade['name'] for celebridade in analise_de_celebridades.result['celebrities']]
                print(lista_celebridades)
                
                descricacao = client.describe_image(url,3,"en")
                texto_descricao = descricacao.captions[0].text

                print("Descricao :", texto_descricao)

                analise_de_imagem = client.analyze_image(url, visual_features = [VisualFeatureTypes.tags])
                lista_tags = [tag.name for tag in analise_de_imagem.tags]
                
                print("Tag: ", lista_tags)

                resultados = {
                    'usuario': tweet.user.screen_name,
                    'texto': status.text,
                    'url': url,
                    'celebridades': lista_celebridades,
                    'descricao': texto_descricao,
                    'tags': lista_tags
                }

                with open('tweets.txt', 'a') as arquivo:
                    arquivo.write(json.dumps(resultados))
                    arquivo.write('\n')

            print('----------------------')
            print('\n\n')
            
    except KeyboardInterrupt:
        None


In [ ]:
minhaStream = None
minhaStream = tw.Stream(auth = auth, listener=MinhaStreamListener())

In [ ]:
minhaStream.filter(follow=[''])

# Avaliando resultados

In [ ]:
lista_arquivo = []
with open('tweets.txt', 'r') as arquivo:
    for linha in arquivo:
        lista_arquivo.append(json.loads(arquivo))

In [ ]:
import pandas as pd

In [ ]:
imagens = pd.DataFrame(lista_arquivo)
imagens.head()

In [ ]:
pesquisa = 'concert'

achados = [(pesquisa in lista) for lista in imagens.tags]

In [ ]:
imagens[achados].descricao.iloc[0]

In [ ]:
imagens[achados].tags.iloc[0]

In [ ]:
imagens[achados].url.iloc[0]